In [30]:
using Dates
using ArgParse
using Printf
using Random

include("../btg.jl")
include("../datasets/load_abalone.jl");

In [31]:
# shuffle data
ind_shuffle = randperm(MersenneTwister(1234), size(data, 1)) 
data = data[ind_shuffle, :]
target = target[ind_shuffle]
id_train = 1:100; posx = 1:7; posc = 1:3
x = data[id_train, posx] 
Fx = data[id_train, posc] 
y = float(target[id_train])
trainingData1 = trainingData(x, Fx, y) #training data used for testing various functions
d = getDimension(trainingData1); n = getNumPts(trainingData1); p = getCovDimension(trainingData1)

3

In [32]:
#parameter setting
rangeθ = [10.0 1000]
rangeθm = repeat(rangeθ, d, 1)
rangeλ = [-1. 1.] #we will always used 1 range scale for lambda
myquadtype = ["SparseGrid", "Gaussian"]
btg1 = btg(trainingData1, rangeθm, rangeλ; quadtype = myquadtype)
(pdf, cdf, dpdf, quantInfo) = solve(btg1); 

In [33]:
id_test = 1001:1002
n_test = length(id_test)
Fx0 = reshape(data[id_test, posc], n_test, length(posc))
x0 = reshape(data[id_test, posx], n_test, length(posx)) 
count_test = 0
error_abs = 0.
error_sq = 0.
nlpd = 0.

0.0

In [42]:
for i in 1:n_test
    x0_i = reshape(x0[i, :], 1, length(posx))
    Fx0_i = reshape(Fx0[i, :], 1, length(posc))
    pdf_i, cdf_i, dpdf_i, quantbound_i, support_i = pre_process(x0_i, Fx0_i, pdf, cdf, dpdf, quantInfo)
    median_i = quantile(cdf_i, quantbound_i, support_i)[1]
    CI_i = credible_interval(cdf_i, quantbound_i, support_i; mode=:equal, wp=.95)[1]
    y_i_true = getLabel(btg1.trainingData)[i]
    count_test += (y_i_true >= CI_i[1])&&(y_i_true <= CI_i[2]) ? 1 : 0
    error_abs += abs(y_i_true - median_i)
    error_sq += (y_i_true - median_i)^2
    nlpd += log(pdf_i(y_i_true)) 
    @info "Current info " i, y_i_true, median_i, CI_i
end

┌ Info: Current info 
│   (i, y_i_true, median_i, CI_i) = (1, 0.27586206896551724, 0.23442142569475435, [0.17424248579279777, 0.3227448459374674])
└ @ Main In[42]:12
┌ Info: Current info 
│   (i, y_i_true, median_i, CI_i) = (2, 0.3448275862068966, 0.25659293473895267, [0.10485234232103113, 0.6555299410868783])
└ @ Main In[42]:12
